In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler

# 그래프 외부에 출력
%matplotlib tk

tf.set_random_seed(777)  # 어느 컴퓨터에서 이 코드를 실행해도 학습 방향이 같도록, 다시 수행해도 같도록

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 4
data_dim = 4
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 892

scaler = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화
scaler2 = MinMaxScaler(feature_range=(0, 1)) # 데이터 일반화

# xy = np.loadtxt('./v3data/train_v3_data_cct.csv', delimiter=',')
# cct, cas_swr, 446to477, uvb, temp, ptmt (892행 5열)
xy = np.loadtxt('./v4data/train_v4_data.csv', delimiter=',')
x = scaler.fit_transform(xy[:, 0:-1]) # x = 맨 마지막 ptmt 제외 모든 것
y = scaler2.fit_transform(xy[:, [-1]])  # y = ptmt

# build a dataset
dataX = []
dataY = []

# print(len(x))
# print(len(y))
# print(x[0])
# print(y[0])

for i in range(0, len(x) - seq_length): # 한 행씩 dataX, Y에 추가
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)
#     print(np.shape(_x))

# print(np.shape(dataX))
# print(np.shape(dataY))

data = '2018-02-27'
# xy2 = np.loadtxt('./v3data/'+ data +'.csv',delimiter=',')
# xy2 = np.loadtxt('./v3data/test_v3_data_cct_180331.csv', delimiter=',')
xy2 = np.loadtxt('./v4data/test_v4_data_180227.csv', delimiter=',') # 예측할 날짜

# x2 = scaler.fit_transform(xy2) # 데이터 없을때 (데이터란 ptmt 값)
x2 = scaler.fit_transform(xy2[:, 0:-1]) # 데이터 있을때
y2 = scaler2.fit_transform(xy2[:, [-1]]) #데이터 있을때

# print(np.shape(x2))

dataX2 = []
dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
    _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
    dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

# print(train_size)
# print(test_size)

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

# print(np.shape(trainX))

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
    dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 3435.13818359375
[step: 1] loss: 2961.3837890625
[step: 2] loss: 2521.8017578125
[step: 3] loss: 2112.0439453125
[step: 4] loss: 1731.340087890625
[step: 5] loss: 1381.1416015625
[step: 6] loss: 1064.47607421875
[step: 7] loss: 785.85888671875
[step: 8] loss: 551.671875
[step: 9] loss: 370.0962829589844
[step: 10] loss: 249.84840393066406
[step: 11] loss: 196.5001678466797
[step: 12] loss: 204.7476043701172
[step: 13] loss: 250.81629943847656
[step: 14] loss: 300.6269226074219
[step: 15] loss: 328.70263671875
[step: 16] loss: 326.30242919921875
[step: 17] loss: 297.65966796875
[step: 18] loss: 252.90843200683594
[step: 19] loss: 202.72052001953125
[step: 20] loss: 155.48779296875
[step: 21] loss: 116.39830780029297
[step: 22] loss: 87.63726043701172
[step: 23] loss: 69.11192321777344
[step: 24] loss: 59.30617904663086
[step: 25] loss: 56.03329849243164
[step: 26] loss: 56.99007797241211
[step: 27] loss: 60.10298156738281
[step: 28] loss: 63.70323944091797
[step: 29] los

[step: 230] loss: 1.3904385566711426
[step: 231] loss: 1.3864455223083496
[step: 232] loss: 1.382529377937317
[step: 233] loss: 1.3786883354187012
[step: 234] loss: 1.374922275543213
[step: 235] loss: 1.3712284564971924
[step: 236] loss: 1.3676055669784546
[step: 237] loss: 1.3640518188476562
[step: 238] loss: 1.3605663776397705
[step: 239] loss: 1.3571480512619019
[step: 240] loss: 1.3537944555282593
[step: 241] loss: 1.350504755973816
[step: 242] loss: 1.347277045249939
[step: 243] loss: 1.3441107273101807
[step: 244] loss: 1.3410041332244873
[step: 245] loss: 1.3379563093185425
[step: 246] loss: 1.3349655866622925
[step: 247] loss: 1.3320304155349731
[step: 248] loss: 1.3291505575180054
[step: 249] loss: 1.3263237476348877
[step: 250] loss: 1.3235491514205933
[step: 251] loss: 1.3208262920379639
[step: 252] loss: 1.3181531429290771
[step: 253] loss: 1.3155288696289062
[step: 254] loss: 1.3129523992538452
[step: 255] loss: 1.3104227781295776
[step: 256] loss: 1.307938575744629
[step:

[step: 458] loss: 1.1055810451507568
[step: 459] loss: 1.1049299240112305
[step: 460] loss: 1.104279637336731
[step: 461] loss: 1.1036299467086792
[step: 462] loss: 1.1029807329177856
[step: 463] loss: 1.102331519126892
[step: 464] loss: 1.1016836166381836
[step: 465] loss: 1.1010360717773438
[step: 466] loss: 1.100388526916504
[step: 467] loss: 1.0997415781021118
[step: 468] loss: 1.0990949869155884
[step: 469] loss: 1.098449468612671
[step: 470] loss: 1.0978039503097534
[step: 471] loss: 1.0971590280532837
[step: 472] loss: 1.0965144634246826
[step: 473] loss: 1.0958702564239502
[step: 474] loss: 1.0952268838882446
[step: 475] loss: 1.094583511352539
[step: 476] loss: 1.0939406156539917
[step: 477] loss: 1.0932986736297607
[step: 478] loss: 1.0926568508148193
[step: 479] loss: 1.0920155048370361
[step: 480] loss: 1.0913745164871216
[step: 481] loss: 1.0907344818115234
[step: 482] loss: 1.0900943279266357
[step: 483] loss: 1.0894546508789062
[step: 484] loss: 1.0888152122497559
[step:

[step: 685] loss: 0.9731251001358032
[step: 686] loss: 0.9726645350456238
[step: 687] loss: 0.9722058773040771
[step: 688] loss: 0.9717493057250977
[step: 689] loss: 0.9712944030761719
[step: 690] loss: 0.9708419442176819
[step: 691] loss: 0.9703912734985352
[step: 692] loss: 0.9699423909187317
[step: 693] loss: 0.9694954752922058
[step: 694] loss: 0.9690502285957336
[step: 695] loss: 0.9686080813407898
[step: 696] loss: 0.9681674242019653
[step: 697] loss: 0.9677286744117737
[step: 698] loss: 0.9672918319702148
[step: 699] loss: 0.9668571352958679
[step: 700] loss: 0.966424822807312
[step: 701] loss: 0.965993344783783
[step: 702] loss: 0.96556556224823
[step: 703] loss: 0.9651392102241516
[step: 704] loss: 0.9647149443626404
[step: 705] loss: 0.9642931222915649
[step: 706] loss: 0.963873565196991
[step: 707] loss: 0.9634553790092468
[step: 708] loss: 0.9630399942398071
[step: 709] loss: 0.9626261591911316
[step: 710] loss: 0.9622151851654053
[step: 711] loss: 0.9618062376976013
[step:

In [6]:
# Plot predictions
plt.title(data + ' predict')
plt.plot(dataY2, label='row data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
# plt.ylabel("uvb")
# plt.ylabel("cct")
plt.ylabel("swr")
plt.xticks(np.arange(0, 892, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
# plt.yticks(np.arange(0, 1.5, step=0.2))
plt.ylim(0, 50)
# plt.yticks(np.arange(0, 60000, step=10000))
plt.grid(True)
plt.legend(loc='upper right')
plt.show()